# Downtown Brooklyn Taxi Trips Study - Step-by-Step
This notebook analyzes taxi trip data, mapping pickups and dropoffs to street segments and visualizing counts.

In [ ]:
import urban_mapper as um

# Initialise UrbanMapper
mapper = um.UrbanMapper()

# Step 1: Create urban layer for street segments
layer = (
    mapper.urban_layer
    .with_type("streets_roads")
    .from_place("Downtown Brooklyn, New York City, USA", network_type="drive")
    .build()
)

In [ ]:
# Step 2: Load taxi trip data
data = (
    mapper.loader
    .from_file("./taxisvis1M.csv")
    .with_columns(longitude_column="pickup_longitude", latitude_column="pickup_latitude")
    .load()
)

In [ ]:
# Step 3: Impute missing coordinates
imputer_pickup = (
    mapper.imputer
    .with_type("SimpleGeoImputer")
    .on_columns("pickup_longitude", "pickup_latitude")
    .build()
)
data = imputer_pickup.transform(data, layer)

imputer_dropoff = (
    mapper.imputer
    .with_type("SimpleGeoImputer")
    .on_columns("dropoff_longitude", "dropoff_latitude")
    .build()
)
data = imputer_dropoff.transform(data, layer)

In [ ]:
# Step 4: Filter to bounding box
filter_step = mapper.filter.with_type("BoundingBoxFilter").build()
data = filter_step.transform(data, layer)

In [ ]:
# Step 5: Map pickups and dropoffs
_, mapped_pickups = layer.map_nearest_layer(
    data,
    longitude_column="pickup_longitude",
    latitude_column="pickup_latitude",
    output_column="pickup_segment"
)
_, mapped_dropoffs = layer.map_nearest_layer(
    data,
    longitude_column="dropoff_longitude",
    latitude_column="dropoff_latitude",
    output_column="dropoff_segment"
)

In [ ]:
# Step 6: Enrich with counts
enricher_pickup = (
    mapper.enricher
    .with_data(group_by="pickup_segment")
    .count_by(output_column="pickup_count")
    .build()
)
enriched_layer_pickup = enricher_pickup.enrich(mapped_pickups, layer)

enricher_dropoff = (
    mapper.enricher
    .with_data(group_by="dropoff_segment")
    .count_by(output_column="dropoff_count")
    .build()
)
enriched_layer = enricher_dropoff.enrich(mapped_dropoffs, enriched_layer_pickup)

In [ ]:
# Step 7: Visualize interactively
visualiser = (
    mapper.visual
    .with_type("Interactive")
    .with_style({"tiles": "CartoDB dark_matter"})
    .build()
)
fig = visualiser.render(enriched_layer.get_layer(), columns=["pickup_count", "dropoff_count"])
fig